# B3 Stock Latest Trade getter function

In [0]:

%sh
pip install -q -U pip
pip install -q -U -r requirements.txt

In [0]:
from databricks_langchain import uc_ai

In [0]:
CATALOG = "<REDACTED>"
SCHEMA = "<REDACTED>"

In [0]:
def get_latest_trade(ticker: str) -> dict[str, str]:
    """
    Get the latest available trading record for a B3 stock.

    Args:
        ticker (str): The stock ticker symbol (without .SA suffix)

    Returns:
        dict: A dictionary containing the latest trading data or None if not found.
            All values are returned as strings to enforce compliance with UC Funtions.
            From their docs: "For container types like `list`, `tuple` and `dict`,
            the inner types **must be specified** and must be uniform (Union types
            are not permitted)".

    Example:
        >>> get_latest_trade("PETR4")
        {
            "date": "2025-06-20"
            "open": "32.72999954223633"
            "high": "33.15999984741211"
            "low": "32.630001068115234"
            "close": "32.81999969482422"
            "volume": "84911200.0"
        }
    """
    # From the UC docs (https://docs.unitycatalog.io/ai/client): Import statements
    # should be contained within the function body (using local imports) to ensure
    # that the dependency is available for use by your function's internal logic.
    import yfinance as yf

    try:
        # Append .SA suffix for B3 stocks
        b3_ticker = f"{ticker.upper()}.SA"

        # Get the stock info
        stock = yf.Ticker(b3_ticker)

        # Get today's data (or last available trading day)
        hist = stock.history(period="1d")

        if hist.empty:
            return {}

        # Get the latest record
        latest = hist.iloc[-1]

        return {
            "date": latest.name.strftime("%Y-%m-%d"),
            "open": str(latest["Open"]),
            "high": str(latest["High"]),
            "low": str(latest["Low"]),
            "close": str(latest["Close"]),
            "volume": str(latest["Volume"]),
        }

    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return {}

In [0]:
uc_client = uc_ai.DatabricksFunctionClient()
uc_ai.set_uc_function_client(uc_client)

# Create the function in the Unity Catalog catalog and schema specified.
# When using `.create_python_function`, the provided function's metadata
# (docstring, parameters, return type) are used to create a tool in the
# specified catalog and schema.
function_info = uc_client.create_python_function(
  func=get_latest_trade,
  catalog=CATALOG,
  schema=SCHEMA,
  replace=True,  # Set to True to overwrite if the function already exists.
)